In [ ]:
inpath="/Users/anthonytownsend/Dropbox/Desktop/_code/notebooks/pyspark/data/in/njtransit_bus_singles_grabs_for_bundling"
outpath="/Users/anthonytownsend/Dropbox/Desktop/_code/notebooks/pyspark/data/out/njtransit_bus_singles_grabs_bundled"

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType, LongType

spark = SparkSession.builder \
  .master("local") \
  .appName("partioning_project") \
  .config("spark.executor.cores", 3) \
  .getOrCreate()

## probably wont need this stuff

In [ ]:
# # option 3, user-defined schema
# schema = StructType() \
#       .add("pkey",StringType(),True) \
#       .add("lat",DoubleType(),True) \
#       .add("lon",DoubleType(),True) \
#       .add("cars",StringType(),True) \
#       .add("consist",StringType(),True) \
#       .add("d",StringType(),True) \
#       .add("dn",StringType(),True) \
#       .add("fs",StringType(),True) \
#       .add("id",StringType(),True) \
#       .add("m",StringType(),True) \
#       .add("op",StringType(),True) \
#       .add("pd",StringType(),True) \
#       .add("pdrtpifeedname",StringType(),True) \
#       .add("pid",StringType(),True) \
#       .add("rt",StringType(),True) \
#       .add("rtrtpifeedname",StringType(),True) \
#       .add("rtdd",StringType(),True) \
#       .add("rtpifeedname",StringType(),True) \
#       .add("run",StringType(),True) \
#       .add("wid1",StringType(),True) \
#       .add("wid2",StringType(),True) \
#       .add("timestamp",StringType(),True)
#       # .add("timestamp",TimestampType(),True)
#       # Spark doesn't recognize date64 types, and no way to create user-defined type
#       # possibly a solution here we can implement after the import by mapping the timestamp column? 
#       # https://arrow.apache.org/docs/python/timestamps.html


# df = spark.read.csv(infile, header = True, schema = schema)

In [ ]:
# # https://sparkbyexamples.com/spark/pyspark-to_timestamp-convert-string-to-timestamp-type/
# from pyspark.sql.functions import *

# #Timestamp String to DateType
# new_df = df.withColumn("timestamp",to_timestamp("timestamp"))

### compute the date component columns on the fly as we write the partitioned files
### 1 file per partition

In [ ]:
# write the 
new_df \
    .withColumn("year", year(col("timestamp"))) \
    .withColumn("month", month(col("timestamp"))) \
    .withColumn("day", dayofmonth(col("timestamp"))) \
    .withColumn("hour", hour(col("timestamp"))) \
    .repartition("year", "month", "day", "hour") \
    .write \
    .mode('overwrite')\
    .partitionBy("year", "month", "day", "hour") \
    .parquet(outpath)